In [ ]:
from zipfile import ZipFile
with ZipFile("main_dataset_folder.zip", "r") as zip_ref:
  zip_ref.extractall("./")
print("Extraction of zip file complete")

In [ ]:
from datasets import  load_dataset, Audio

dataset = load_dataset(
    "csv",
    data_files="/content/main_dataset_folder/main_dataset.csv",
    split = "train"
)

dataset

In [ ]:
dataset = dataset.remove_columns(["Unnamed: 0", "id", "filename", "start", "end", "duration"])

In [ ]:
dataset

In [ ]:
import os

AUDIO_ROOT = "/main_dataset_folder"

def fix_path(example):
  example["file_path"] = os.path.join(AUDIO_ROOT, example["file_path"])
  return example

dataset = dataset.map(fix_path)